# 1. Set Working Directories and Dependencies
Define handy paths for the ARC cluster and local workspace, then import everything needed for remote file transfers, data loading, and plotting.

In [ ]:
from __future__ import annotations

from pathlib import Path
import getpass
import subprocess

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import paramiko
import pypsa
import xarray as xr

REMOTE_USER = "engs2523"
REMOTE_HOST = "arc-login.arc.ox.ac.uk"
REMOTE_ROOT = Path("/data/engs-df-green-ammonia/engs2523/pypsa-earth")
RUN_NAME = "europe"
REMOTE_RESULTS = REMOTE_ROOT / "results" / RUN_NAME
LOCAL_ROOT = Path.cwd().resolve()
LOCAL_RESULTS = LOCAL_ROOT / "data_remote" / RUN_NAME
LOCAL_RESULTS.mkdir(parents=True, exist_ok=True)

print(f"Remote results path: {REMOTE_RESULTS}")
print(f"Local download target: {LOCAL_RESULTS}")

# 2. Verify Remote Snakemake Outputs
List the remote result files via an SSH call so we only download once the expected artifacts exist.

In [ ]:
def run_remote(command: str) -> subprocess.CompletedProcess:
    full_cmd = [
        "ssh",
        f"{REMOTE_USER}@{REMOTE_HOST}",
        command,
    ]
    result = subprocess.run(full_cmd, capture_output=True, text=True, check=False)
    if result.stdout:
        print(result.stdout)
    if result.stderr:
        print(result.stderr)
    if result.returncode != 0:
        raise RuntimeError(f"Remote command failed: {command}\n{result.stderr}")
    return result

run_remote(f"cd {REMOTE_RESULTS} && ls -R | head -n 80")

# 3. Download Results to Local Machine
Use `rsync` (or `scp`) from within the notebook to mirror the remote `results/europe` folder into `data_remote/europe/` for offline analysis.

In [ ]:
def sync_results(rsync_flags: list[str] | None = None) -> None:
    target_dir = LOCAL_RESULTS
    target_dir.mkdir(parents=True, exist_ok=True)
    flags = rsync_flags or ["-av", "--progress"]
    cmd = [
        "rsync",
        *flags,
        f"{REMOTE_USER}@{REMOTE_HOST}:{REMOTE_RESULTS}/",
        str(target_dir),
    ]
    print("Running:", " ".join(cmd))
    subprocess.run(cmd, check=True)

sync_results()
print("Local files:")
for path in sorted(LOCAL_RESULTS.rglob("*")):
    if path.is_file():
        print(" -", path.relative_to(LOCAL_RESULTS))

# 4. Load Metadata and Geometry
Read the solved PyPSA network plus any helper geometry (e.g., the built-in `resources/regions/onshore_shapes.geojson`) so we can merge KPIs with shapes.

In [ ]:
network_path = LOCAL_RESULTS / "networks" / "elec_s_37_ec_lcopt_Co2L-3h.nc"
if not network_path.exists():
    raise FileNotFoundError(f"Network results missing: {network_path}")

n = pypsa.Network(network_path)
print(f"Snapshots: {n.snapshots}")
print("Total load (MWh):", n.loads_t.p.sum().sum())
print("Objective value (EUR):", n.objective)

carrier_capacity = (
    n.generators.groupby("carrier")["p_nom_opt"].sum().sort_values(ascending=False)
 )

# 5. Visualize Results on Map
Merge KPIs (marginal prices, generation build) with the geometry and produce a quick map using both GeoPandas and `pypsa.Network.plot`.

In [ ]:
bus_df = n.buses.reset_index().rename(columns={"index": "bus"})
prices = n.buses_t.marginal_price.iloc[0]
bus_df["marginal_price"] = bus_df["bus"].map(prices)
bus_df["p_nom_opt"] = n.generators.groupby("bus")["p_nom_opt"].sum()
bus_gdf = gpd.GeoDataFrame(
    bus_df,
    geometry=gpd.points_from_xy(bus_df.x, bus_df.y),
    crs="EPSG:4326",
)

fig, ax = plt.subplots(figsize=(8, 8))
region_shapes.boundary.plot(ax=ax, linewidth=0.5, color="gray")
scatter = bus_gdf.plot(
    ax=ax,
    column="marginal_price",
    cmap="viridis",
    markersize=(bus_gdf["p_nom_opt"].fillna(0) / 500).clip(5, 400),
    legend=True,
    legend_kwds={"label": "€/MWh"},
)
ax.set_title("Bus marginal price and built capacity (marker size)")
ax.set_axis_off()
plt.show()

fig, ax = plt.subplots(figsize=(8, 8))
line_widths = (n.lines.s_nom_opt / n.lines.s_nom_opt.max()).fillna(0) * 5
n.plot(
    ax=ax,
    bus_sizes=2e3,
    bus_colors=prices,
    bus_cmap="viridis",
    line_widths=line_widths,
    title="PyPSA-Earth Europe network snapshot",
)
plt.show()